In [1]:
import re

import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# load dataset
train_df = pd.read_csv("data/train.csv")

# define parameters
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train_df["text"].values)
X = tokenizer.texts_to_sequences(train_df["text"].values)
X = pad_sequences(X)

In [3]:
# compose LSTM network
num_class = 3
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(rate=0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_class, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 746, 128)          256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 746, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
Y = pd.get_dummies(train_df["author"]).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

((13117, 746), (13117, 3))
((6462, 746), (6462, 3))


In [5]:
BATCH_SIZE = 128
LR = 0.001
DC = 0.1
EPOCH=20

callbacks = [EarlyStopping(monitor="loss", min_delta=0.001, patience=3),
             LearningRateScheduler(lambda epoch: LR/(1 + DC*epoch))]
model.fit(X_train, Y_train, epochs=EPOCH, batch_size=BATCH_SIZE,
          validation_data=(X_test, Y_test),
          callbacks=callbacks)

Train on 13117 samples, validate on 6462 samples
Epoch 1/20
13117/13117 [==============================] - 154s - loss: 0.9094 - acc: 0.5700 - val_loss: 0.6642 - val_acc: 0.7262
Epoch 2/20
13117/13117 [==============================] - 155s - loss: 0.5704 - acc: 0.7685 - val_loss: 0.5618 - val_acc: 0.7652
Epoch 3/20
13117/13117 [==============================] - 145s - loss: 0.4737 - acc: 0.8107 - val_loss: 0.5775 - val_acc: 0.7617
Epoch 4/20
13117/13117 [==============================] - 140s - loss: 0.4322 - acc: 0.8271 - val_loss: 0.5646 - val_acc: 0.7700
Epoch 5/20
13117/13117 [==============================] - 140s - loss: 0.4005 - acc: 0.8423 - val_loss: 0.5780 - val_acc: 0.7693
Epoch 6/20
13117/13117 [==============================] - 138s - loss: 0.3831 - acc: 0.8459 - val_loss: 0.5808 - val_acc: 0.7654
Epoch 7/20
13117/13117 [==============================] - 139s - loss: 0.3680 - acc: 0.8550 - val_loss: 0.6007 - val_acc: 0.7676
Epoch 8/20
13117/13117 [========================